In [1]:
# Import libaries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
final_list = []

# Skytrax URL
url = 'https://www.airlinequality.com/airline-reviews/myanmar-national-airlines/?sortby=post_date%3ADesc&pagesize=20'

# Request URL
response = requests.get(url)

# Create Soup
soup = BeautifulSoup(response.content, 'lxml')

# For loop for HTML
for idx, review in enumerate(soup.find_all('div',{'class':"review-stats"})):
    
    # Create a empty dictionary to contain columns and values
    review_dict = dict()
    
    # Values that is in star fill
    names = review.find_all('td', class_='review-rating-header')[::-1]
    ratings = review.find_all('td', class_='stars')[::-1]
    
    try:
        for name, rating in zip(names[1:], ratings):
            review_dict[name.text] = rating.find_all(class_='fill')[-1].text
    except:
        review_dict[name.text] = np.nan
    
    # Author, Date, Overall_rating,Text_content, Recommended
    review_dict['Author'] = soup.find_all(itemprop='name')[idx+1].text
    review_dict['Date'] = soup.find_all(itemprop='datePublished')[::2][idx].attrs['content']
    review_dict['text_content'] = soup.find_all('div',class_='text_content')[idx].text
    review_dict['Recommended'] = soup.find_all('div',{'class':"review-stats"})[idx].find_all('tr')[-1].find_all('td')[-1].text
        
    # Type of Traveller
    try:
        review.find('td',{'class':'review-rating-header type_of_traveller'}).text is not None
        review_dict['traveller_type'] = soup.find_all('td',{'class':'review-rating-header type_of_traveller'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['traveller_type'] = np.nan
    
    # Seat type
    try:
        review.find('td',{'class':'review-rating-header cabin_flown'}).text is not None
        review_dict['seat_type'] = soup.find_all('td',{'class':'review-rating-header cabin_flown'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['seat_type'] = np.nan
    
    try:
        review_dict['Overall_rating'] = soup.find_all('div',{'itemprop':'reviewRating'})[idx].find(itemprop="ratingValue").text
    
    except:
        review_dict['Overall_rating'] = np.nan
    
    
    # Append dictionary into a list
    final_list.append(review_dict)

# Create DataFrame
df = pd.DataFrame(final_list)
df['airline'] = 'myanmar national airlines'

# rearrange dataframe
df = df[['airline','Author','Date','text_content','traveller_type','seat_type','Overall_rating','Seat Comfort',
 'Cabin Staff Service','Food & Beverages','Inflight Entertainment','Ground Service','Value For Money',  
  'Recommended']]

# rename dataframe
df.columns = ['airline_name', 'author', 'date', 'content', 'type_traveller','cabin_flown', 'overall_rating', 'seat_comfort_rating',
'cabin_staff_rating', 'food_beverages_rating','inflight_entertainment_rating', 'ground_service_rating','value_money_rating', 'recommended']

# Save to CSV
df.to_csv('Data/myanmar.csv',index=False)

In [3]:
# Sanity check
df.tail()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
6,myanmar national airlines,Min Htet,2017-12-08,✅ Trip Verified | Yangon to Mandalay. Lounge ...,Solo Leisure,Business Class,10,5,5,5,5,5,5,yes
7,myanmar national airlines,Even Nordhagen,2017-06-06,✅ Verified Review | Singapore to Yangon. I di...,Solo Leisure,Economy Class,10,5,5,5,5,5,5,yes
8,myanmar national airlines,A Green,2017-05-05,"✅ Verified Review | Very happy with my trip, ...",Solo Leisure,Economy Class,10,4,5,4,NaN,4,3,yes
9,myanmar national airlines,F Marks,2017-02-05,✅ Verified Review | Flew the quick 45 minute f...,Couple Leisure,Economy Class,8,4,5,4,1,2,2,yes
10,myanmar national airlines,An Thung,2016-08-15,✅ Verified Review | I traveled Yangon to Sing...,Solo Leisure,Economy Class,7,4,4,3,4,4,5,yes


In [4]:
df.head()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
0,myanmar national airlines,Michael Schade,2019-05-13,✅ Trip Verified | Singapore to Thandwe via Yan...,Solo Leisure,Economy Class,10,4,5,5,NaN,4,4,yes
1,myanmar national airlines,Thomas van Woerkom,2019-03-21,✅ Trip Verified | Yangon to Heho with Myanmar ...,Couple Leisure,Economy Class,8,5,3,4,NaN,2,4,yes
2,myanmar national airlines,G. Stoykovich,2018-12-28,"✅ Trip Verified | Loikaw to Yangon. Quick, ef...",Couple Leisure,Economy Class,10,5,5,5,NaN,5,5,yes
3,myanmar national airlines,Wai Lynn Htun,2018-10-04,✅ Trip Verified | Yangon to Mandalay. Aircraf...,Business,Economy Class,1,3,3,NaN,NaN,1,1,no
4,myanmar national airlines,M Andrews,2018-06-11,✅ Trip Verified | The flight was about one hou...,Family Leisure,Economy Class,9,4,5,4,NaN,5,4,yes
